In [1]:
import pandas as pd
import query_rx as qrx

### Quick Start Overview
The entire NDC to ATC mapping can be run with a single function -- the only input is a list of NDC11 codes (as strings) which we would like to map.  

The NDC to ATC mapping is accomplished through the RXCUI identifier `NDC11 -> RXCUI -> ATC`, so the mapping requires two API queries.
- One to map NDC to RXCUI
- A second to map RXCUI to ATC

The output is `data/clean/ndc_to_atc.pkl` a file containing the mapping from NDC to the ATC classes along with several other pieces of information grapped from the RxNorm API.

In [2]:
ndc11s = ['00002759701', '00169450101']
qrx.main_query_from_new_ndc11(ndc11s)

Querying an additional 2 NDC codes
Querying an additional 2 RXCUI codes


After the function has been run, if it is run again it will only query the API for new NDCs and RXCUIs it doesn't already have data for. In the case below we have an unique NDC at the end of the list so it will query the API for that NDC. But because this NDC will have the exact same RXCUI as the second NDC in the list (already queried) the second query does not need to run.

In [3]:
# Adding one new NDC11 to the end of the same list, method will ignore NDCs it already has data on. 
ndc11s = ['00002759701', '00169450101', '00169450114']
qrx.main_query_from_new_ndc11(ndc11s)

Querying an additional 1 NDC codes
Querying an additional 0 RXCUI codes


In [4]:
df = pd.read_pickle(qrx.Params().clean_path+'ndc_to_atc.pkl')
df.head().T

,0,1,2
drug_ndc,00002759701,00169450101,00169450114
ndc10,0002-7597-01,0169-4501-01,0169-4501-14
ndc9,0002-7597,0169-4501,0169-4501
rxcui,485968,2553803,2553803
splSetIdItem,,ee06186f-2aa3-4990-a760-757579d8f77b,ee06186f-2aa3-4990-a760-757579d8f77b
packaging_0,1 VIAL in 1 CARTON (0002-7597-01) / 2 mL in 1...,"0.5 mL in 1 SYRINGE, PLASTIC","4 SYRINGE, PLASTIC in 1 CARTON (0169-4501-14) ..."
LABELER,Eli Lilly and Company,Novo Nordisk,Novo Nordisk
LABEL_TYPE,HUMAN PRESCRIPTION DRUG,HUMAN PRESCRIPTION DRUG,HUMAN PRESCRIPTION DRUG
MARKETING_CATEGORY,NDA,NDA,NDA
MARKETING_EFFECTIVE_TIME_LOW,20040401,20210605,20210605


### Detailed Walkthrough
The `query_rx` module is provided several wrapper functions which execute the queries to the RxNorm API. These functions check for whether information has already been scraped so that subsequent method calls only add information for new NDCs. 

To perform the mapping from NDC11 to ATC query RxNorm API to RXCUI: `NDC11 -> RXCUI -> ATC`

#### Step 1:
The first step in the pipeline is to query RxNorm based on the provided NDCs. This returns a DataFrame which most importantly maps the provided 11-digit NDC, `ndcItem` to the `rxcui` label.

In [5]:
df_ndc = qrx.query_RxNorm_ndc_properties(ndc11s)
df_ndc

,ndc10,ndc9,ndcItem,rxcui,splSetIdItem,packaging_0,LABELER,LABEL_TYPE,MARKETING_CATEGORY,MARKETING_EFFECTIVE_TIME_LOW,MARKETING_STATUS,NDA,DM_SPL_ID
0,0002-7597-01,0002-7597,00002759701,485968,,1 VIAL in 1 CARTON (0002-7597-01) / 2 mL in 1...,Eli Lilly and Company,HUMAN PRESCRIPTION DRUG,NDA,20040401,ACTIVE,NDA021253,NaN
1,0169-4501-01,0169-4501,00169450101,2553803,ee06186f-2aa3-4990-a760-757579d8f77b,"0.5 mL in 1 SYRINGE, PLASTIC",Novo Nordisk,HUMAN PRESCRIPTION DRUG,NDA,20210605,ACTIVE,NDA215256,780690
2,0169-4501-14,0169-4501,00169450114,2553803,ee06186f-2aa3-4990-a760-757579d8f77b,"4 SYRINGE, PLASTIC in 1 CARTON (0169-4501-14) ...",Novo Nordisk,HUMAN PRESCRIPTION DRUG,NDA,20210605,ACTIVE,NDA215256,780690


#### Step 2:
The second step is to take the unique rxcuis from the previous query and determine the ATC classes. This returns a dataframe which has the `rxcui` and also the full `ATC` label for that class. 

In [6]:
rxcuis = df_ndc.rxcui.dropna().unique()
df_rxcui = qrx.RxNorm_rxcui_to_atc(rxcuis)
df_rxcui

,rxcui,rxcui_IN,ATC,DRUGBANK,MMSL_CODE,SNOMEDCT,SPL_SET_ID,UNII_CODE,VUID,USP
0,2553803,1991302,A10BJ06,DB13928,IN33346,764284009,fdf509ac-7ae5-49be-9a3e-8465c76f38e1,53AXN4NNHX,4037164,NaN
1,485968,61381,N05AH03,DB00334,IN5198,386849001,fcec190f-b865-c5d3-e053-6394a90a7e45,N7U69T4SZR,4020996,m58486


#### Step 3:
The final step is to merge all of the files together so the final data set effectively links `NDC -> ATC`. The repository contains labeling files for different levels of ATC aggregation and those are appended to the data for ease of use.  

These steps are performed by `raw_to_clean_ndc_to_atc()` which is the final step of the convenience method `main_query_from_new_ndc11`

In [7]:
df = df_ndc.merge(df_rxcui, on='rxcui', how='left')
df = df.rename(columns={'ndcItem': 'drug_ndc'})

# Add in the levels
for level in qrx.Params().atc_levels:
    dfl = pd.read_pickle(qrx.Params().clean_path+f'atc_{level}_level.pkl')
    dfl = dfl.set_index(f'atc_{level}')[f'atc_{level}_desc']
    
    df[f'atc_{level}'] = df['ATC'].str.extract(r'^'+'('+'|'.join(dfl.index)+')')
    df[f'atc_{level}_desc'] = df[f'atc_{level}'].map(dfl)

cols = ['drug_ndc', 'ATC'] + [x for x in df.columns if 'atc' in x]
display(df[cols])

,drug_ndc,ATC,atc_1,atc_1_desc,atc_2,atc_2_desc,atc_3,atc_3_desc,atc_4,atc_4_desc
0,00002759701,N05AH03,N,Nervous System,N05,Psycholeptics,N05A,ANTIPSYCHOTICS,N05AH,"Diazepines, oxazepines, thiazepines and oxepines"
1,00169450101,A10BJ06,A,Alimentary tract and metabolism,A10,Drugs used in diabetes,A10B,"BLOOD GLUCOSE LOWERING DRUGS, EXCL. INSULINS",A10BJ,Glucagon-like peptide-1 (GLP-1) analogues
2,00169450114,A10BJ06,A,Alimentary tract and metabolism,A10,Drugs used in diabetes,A10B,"BLOOD GLUCOSE LOWERING DRUGS, EXCL. INSULINS",A10BJ,Glucagon-like peptide-1 (GLP-1) analogues
